In [1]:
## PREPROCESS
# chia dataset ra
# thêm negative là dự đoán của bge-m3

## HYBRID SEARCH
# dùng BM25 để lấy top 50
# tạo Dataset class phù hợp vói triplet loss
# fine tune bi-encoder PhoBERT triplet loss với LoRA và lấy top 30
# hợp nhất và lấy top 40

## RERANK
# tạo Dataset class phù hợp vói binary classification
# fine tune cross encoder PhoBERT với binary classification

# EVAL
%env CUDA_LAUNCH_BLOCKING=1


# PREPROCESS

In [2]:
import pandas as pd
corpus = pd.read_csv("/kaggle/input/retrieval/corpus.csv")
train = pd.read_csv("/kaggle/input/retrieval/train.csv")

## DATASET

In [3]:
import pandas as pd
import re

def prepare_data(train_df, corpus_df, sample_size=500, max_corpus_size=10000, output_prefix="hybrid_search", random_state=42):

    train_sampled = train_df.sample(n=sample_size, random_state=random_state)


    def robust_clean_cid_string(s):
        if not isinstance(s, str):
            return []
        s = s.replace('[', '').replace(']', '').replace("'", '').replace('"', '').strip()
        s = re.sub(r'\s+', ',', s)
        try:
            return [int(x) for x in s.split(',') if x.strip().isdigit()]
        except Exception as e:
            print(f"⚠️ Lỗi parse cid: {s} → {e}")
            return []

    train_sampled['cid'] = train_sampled['cid'].apply(robust_clean_cid_string)


    available_cids = set(corpus_df['cid'].unique())
    train_sampled = train_sampled[train_sampled['cid'].apply(lambda cids: all(cid in available_cids for cid in cids))]


    all_cids = set(cid for sublist in train_sampled['cid'] for cid in sublist)
    cid_to_text = corpus_df.set_index("cid")['text'].to_dict()
    train_sampled['positive_context'] = train_sampled['cid'].apply(
        lambda cids: "\n".join([cid_to_text.get(cid, "") for cid in cids if cid in cid_to_text])
    )


    corpus_filtered = corpus_df[corpus_df['cid'].isin(all_cids)].copy()
    corpus_sampled = corpus_filtered.sample(n=1000, random_state=42) if len(corpus_filtered) > max_corpus_size else corpus_filtered


    train_sampled = train_sampled.dropna(subset=['question', 'positive_context'])
    train_sampled.to_csv(f"train_{output_prefix}.csv", index=False)
    corpus_sampled.to_csv(f"corpus_{output_prefix}.csv", index=False)

    print(f"✅ Saved: {len(train_sampled)} training rows and {len(corpus_sampled)} corpus rows.")


prepare_data(train, corpus, sample_size=500, output_prefix="hybrid_search", random_state=123)
prepare_data(train, corpus, sample_size=500, output_prefix="rerank", random_state=456)

In [4]:
import pandas as pd
import ast

def check_cid_consistency(train_path, corpus_path):
    train_df = pd.read_csv(train_path)
    corpus_df = pd.read_csv(corpus_path)

    # Chuẩn hóa cột cid từ string → list[int]
    def parse_cid(x):
        if isinstance(x, list):
            return x
        try:
            return ast.literal_eval(x)
        except:
            return []

    train_df['cid'] = train_df['cid'].apply(parse_cid)

    # Tập hợp tất cả cid xuất hiện trong train
    train_cids = set(cid for sublist in train_df['cid'] for cid in sublist)

    # Tập hợp toàn bộ cid trong corpus
    corpus_cids = set(corpus_df['cid'].unique())

    # So sánh
    missing_cids = train_cids - corpus_cids

    if not missing_cids:
        print("✅ Tất cả CID trong train đều có trong corpus.")
    else:
        print(f"Có {len(missing_cids)} CID bị thiếu trong corpus:")
    return missing_cids
check_cid_consistency(train_path="train_rerank.csv", corpus_path="corpus_rerank.csv")
check_cid_consistency(train_path="train_hybrid_search.csv", corpus_path="corpus_hybrid_search.csv")

## thêm hard negative bằng dự đoán của bge-m3

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def generate_hard_negatives_with_bge(
    train_file: str,
    corpus_file: str,
    output_file: str = "train_neg_hybrid_search.csv",
    model_name: str = "BAAI/bge-m3",
    batch_size: int = 64,
    max_length: int = 256
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    # Load data
    train_sampled = pd.read_csv(train_file)
    corpus_sampled = pd.read_csv(corpus_file)

    # Lấy text corpus và map cid
    cid_to_text = corpus_sampled.set_index("cid")['text'].to_dict()
    all_corpus_texts = list(cid_to_text.values())
    all_corpus_cids = list(cid_to_text.keys())

    # Hàm encode
    def get_embedding(texts):
        inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            return outputs.last_hidden_state[:, 0].cpu().numpy()  # [CLS]

    # Embed corpus
    print("🔄 Encoding corpus...")
    corpus_embeddings = []
    for i in tqdm(range(0, len(all_corpus_texts), batch_size)):
        batch_texts = all_corpus_texts[i:i+batch_size]
        batch_emb = get_embedding(batch_texts)
        corpus_embeddings.append(batch_emb)
    corpus_embeddings = np.vstack(corpus_embeddings)

    # Tạo hard negatives
    print("🔍 Generating hard negatives...")
    negatives = []

    for idx, row in tqdm(train_sampled.iterrows(), total=len(train_sampled)):
        question = row['question']
        pos_cids = set(row['cid']) if isinstance(row['cid'], list) else set()

        q_emb = get_embedding([question])
        sims = cosine_similarity(q_emb, corpus_embeddings)[0]
        top_indices = sims.argsort()[::-1]

        selected_neg = None
        for top_idx in top_indices:
            neg_cid = all_corpus_cids[top_idx]
            if neg_cid not in pos_cids:
                selected_neg = all_corpus_texts[top_idx]
                break

        negatives.append(selected_neg or "no hard negative found")

    # Gán và lưu
    train_sampled['negative'] = negatives
    train_sampled = train_sampled.dropna(subset=['question', 'positive_context', 'negative'])
    train_sampled = train_sampled.drop(columns=['positive_context'])
    train_sampled.to_csv(output_file, index=False)

    print(f"✅ Đã tạo negative và lưu: {output_file} ({len(train_sampled)} samples)")
    return train_sampled


In [6]:
generate_hard_negatives_with_bge(
    train_file="train_hybrid_search.csv",
    corpus_file="corpus_hybrid_search.csv",
    output_file="train_neg_hybrid_search.csv"
)

In [7]:
generate_hard_negatives_with_bge(
    train_file="train_rerank.csv",
    corpus_file="corpus_rerank.csv",
    output_file="train_neg_rerank.csv"
)

In [8]:

# Load file
df = pd.read_csv("train_neg_rerank.csv")  # hoặc tên file bạn đang dùng

# Tạo 2 bản ghi cho mỗi dòng: 1 positive và 1 negative
pos_rows = df[["question", "context"]].copy()
pos_rows["label"] = 1
pos_rows = pos_rows.rename(columns={"context": "context"})

neg_rows = df[["question", "negative"]].copy()
neg_rows["label"] = 0
neg_rows = neg_rows.rename(columns={"negative": "context"})

# Gộp lại
rerank_df = pd.concat([pos_rows, neg_rows], ignore_index=True)

# Shuffle để tránh model bias
rerank_df = rerank_df.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Lưu
rerank_df.to_csv("rerank_train.csv", index=False)
print("✅ Saved rerank_train.csv with shape:", rerank_df.shape)


# HYBRID SEARCH

## BM25 để lấy top 20

In [9]:
!pip install rank_bm25

In [10]:
from rank_bm25 import BM25Okapi
def retrieve_bm25(query, corpus, top_k=20):
    corpus = pd.read_csv(corpus)
    bm25 = BM25Okapi(corpus["text"].str.split())
    scores = bm25.get_scores(query.split())
    top_idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    return corpus.iloc[top_idx]

## Bi-encoder để lấy top 10

In [11]:
from torch.utils.data import Dataset
import pandas as pd

class LegalBiEncoderDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row['question']
        pos = row['context']
        neg = row['negative']

        anchor = self.tokenizer(question, padding='max_length', truncation=True,
                                max_length=self.max_length, return_tensors="pt")
        positive = self.tokenizer(pos, padding='max_length', truncation=True,
                                  max_length=self.max_length, return_tensors="pt")
        negative = self.tokenizer(neg, padding='max_length', truncation=True,
                                  max_length=self.max_length, return_tensors="pt")

        return {
            "anchor_input_ids": anchor["input_ids"].squeeze(),
            "anchor_attention_mask": anchor["attention_mask"].squeeze(),
            "positive_input_ids": positive["input_ids"].squeeze(),
            "positive_attention_mask": positive["attention_mask"].squeeze(),
            "negative_input_ids": negative["input_ids"].squeeze(),
            "negative_attention_mask": negative["attention_mask"].squeeze()
        }


In [12]:
import torch
import torch.nn as nn
from transformers import AutoModel
from peft import get_peft_model, LoraConfig, TaskType

class BiEncoderPhoBERT(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        base_model = AutoModel.from_pretrained(model_name)

        # LoRA config
        peft_config = LoraConfig(
            task_type=TaskType.FEATURE_EXTRACTION,
            r=8,
            lora_alpha=16,
            lora_dropout=0.1,
            target_modules=["encoder.layer.11.attention.output.dense"],  # hoặc mở rộng thêm
            bias="none"
        )
        self.encoder = get_peft_model(base_model, peft_config)

    def forward(self, input_ids, attention_mask):
        output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = output.last_hidden_state[:, 0]  # [CLS] vector
        return cls_output


In [13]:
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "vinai/phobert-base"
max_length = 256
epochs = 10
batch_size = 32
lr = 2e-5


tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
dataset = LegalBiEncoderDataset("/kaggle/working/train_neg_hybrid_search.csv", tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = BiEncoderPhoBERT(model_name).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
loss_fn = nn.TripletMarginLoss(margin=0.3)

# Training loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(dataloader):
        anchor = model(batch['anchor_input_ids'].to(device), batch['anchor_attention_mask'].to(device))
        pos = model(batch['positive_input_ids'].to(device), batch['positive_attention_mask'].to(device))
        neg = model(batch['negative_input_ids'].to(device), batch['negative_attention_mask'].to(device))

        loss = loss_fn(anchor, pos, neg)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    
    print(f"Epoch {epoch + 1}: Loss = {total_loss:.4f}")


In [14]:

import os
output_dir = "checkpoints/bi_encoder_phobert"
os.makedirs(output_dir, exist_ok=True)

model.encoder.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ Đã lưu checkpoint tại {output_dir}")

In [15]:
import torch
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel, PeftConfig
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

corpus_df = pd.read_csv("/kaggle/working/corpus_rerank.csv")
corpus_texts = corpus_df["text"].tolist()
corpus_cids = corpus_df["cid"].tolist()

def load_dense_model(checkpoint_path="checkpoints/bi_encoder_phobert"):
    config = PeftConfig.from_pretrained(checkpoint_path)
    base = AutoModel.from_pretrained(config.base_model_name_or_path)
    model = PeftModel.from_pretrained(base, checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, use_fast=False)
    return model.eval(), tokenizer

def encode_corpus(model, tokenizer, device, max_len=256):
    embeddings = []
    model.eval()
    model.to(device)

    with torch.no_grad():
        for text in corpus_texts:
            inputs = tokenizer(text, truncation=True, padding='max_length', max_length=max_len, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            output = model(inputs["input_ids"], inputs["attention_mask"])
            cls_output = output.last_hidden_state[:, 0]  # [CLS] token
            emb = cls_output.cpu().numpy()[0]
            embeddings.append(emb)

    return torch.tensor(embeddings)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, tokenizer = load_dense_model()
corpus_embeddings = encode_corpus(model, tokenizer, device)


def dense_func(query, top_k=10):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(query, truncation=True, padding='max_length', max_length=256, return_tensors="pt").to(device)
        output = model(inputs["input_ids"], inputs["attention_mask"])
        cls_output = output.last_hidden_state[:, 0]  # lấy vector [CLS]
        query_vec = cls_output.cpu().numpy()

    sims = cosine_similarity(query_vec, corpus_embeddings.numpy())[0]
    top_indices = sims.argsort()[::-1][:top_k]

    results = []
    for idx in top_indices:
        results.append({
            "cid": corpus_cids[idx],
            "text": corpus_texts[idx],
            "score": float(sims[idx]) # chỉ để debug thôi, sau xóa
        })

    return results



In [16]:
def hybrid_search(query, bm25_func, dense_func, corpus):
    bm25_res = bm25_func(query, corpus)          
    dense_res = dense_func(query)        

    bm25_list = bm25_res.to_dict(orient="records")

    merged = {item["cid"]: item for item in bm25_list + dense_res}

    # đang ưu tiên bm25
    return list(merged.values())[:25]


### TESTING

In [17]:
# testing
query = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
results = dense_func(query)
results

In [18]:
## testing
query = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
corpus = "/kaggle/working/corpus_hybrid_search.csv"
bm25_results = retrieve_bm25(query,corpus)
bm25_results

In [20]:
## testing
query = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
hybrid_searching = hybrid_search(query, retrieve_bm25, dense_func, corpus )
hybrid_searching

# RERANK


In [21]:
from torch.utils.data import Dataset
import pandas as pd

class RerankDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=256):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row["question"]
        context = row["context"]
        label = row["label"]  # 1 = relevant, 0 = not relevant

        encoded = self.tokenizer(
            question,
            context,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "label": int(label)
        }


In [22]:

os.environ["WANDB_DISABLED"] = "true"
import warnings
warnings.filterwarnings("ignore", message="Be aware, overflowing tokens.*")


In [23]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Load dataset
df = pd.read_csv("rerank_train.csv")  # Cột: question, context, label
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_df.to_csv("rerank_train_split.csv", index=False)
val_df.to_csv("rerank_val_split.csv", index=False)

train_dataset = RerankDataset("rerank_train_split.csv", tokenizer)
val_dataset = RerankDataset("rerank_val_split.csv", tokenizer)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Training args
training_args = TrainingArguments(
    output_dir="./rerank_ckpt",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2
)

# Metric
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

# Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


# Train
trainer.train()


In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

def rerank_with_cross_encoder(question, contexts, model_path, model_name="vinai/phobert-base", top_k=5, device=None, max_length=256):
    import torch
    from transformers import AutoTokenizer, AutoModelForSequenceClassification

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device).eval()


    contexts = [c if isinstance(c, str) else "" for c in contexts]
    contexts = [c[:2048] for c in contexts]  # ngăn tokenizer tạo >512 tokens

    try:
        inputs = tokenizer(
            [question] * len(contexts),
            contexts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(device)
    except Exception as e:
        print("Tokenizer error:", e)
        print("Context length:", [len(c) for c in contexts])
        raise

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)[:, 1]

    results = list(zip(contexts, probs.cpu().tolist()))
    reranked = sorted(results, key=lambda x: x[1], reverse=True)

    return reranked[:top_k]



In [25]:
best_ckpt_path = trainer.state.best_model_checkpoint
print("✅ Best checkpoint path:", best_ckpt_path)


In [26]:
# Một example
question = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
contexts = [item['text'] for item in hybrid_searching]

reranked = rerank_with_cross_encoder(
    question=question,
    contexts=contexts,
    model_path=best_ckpt_path,  # tôi không biết tên checkpoint
    model_name=model_name,
    top_k=3,
    device="cpu"
)

# In kết quả
for i, (ctx, score) in enumerate(reranked, 1):
    print(f"#{i} - score: {score:.4f}")
    print(ctx)
    print("-" * 50)
